In [ ]:
# retrieve top10 documents given the topic
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
import numpy as np
# nltk.download('punkt')
# nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
    stemmer = PorterStemmer()
    tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
    tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
    return [stemmer.stem(word) for word in tokens]


# Files paths

request_file_path = 'Final3_4o_mini.csv'
doc_bank_path = './cwdocs09.tsv'
run_file_path = './sample_runs/dev_bm25_09_original'

# Reads files and build bm25 corpus (index)

dev = pd.read_csv(request_file_path, sep=',')
doc_bank = pd.read_csv(doc_bank_path, sep='\t').fillna('')

# doc_bank['tokenized_doc_list'] = doc_bank['doc'].map(stem_tokenize)
# tokenized_doc_str_list = []
# for d in tqdm(doc_bank['doc']):
#     tokenized_doc_str = stem_tokenize(d)
#     tokenized_doc_str_list.append(tokenized_doc_str)
import pickle
with open('tokenized_09.pkl','rb') as f:
    tokenized_doc_str_list = pickle.load(f)
# with open('tokenized_12.pkl','wb') as f:
#     pickle.dump(tokenized_doc_str_list,f,protocol=1)
doc_bank['tokenized_doc_list']  = tokenized_doc_str_list
doc_bank['tokenized_doc_str'] = doc_bank['tokenized_doc_list'].map(lambda x: ' '.join(x))

bm25_corpus = doc_bank['tokenized_doc_list'].tolist()
bm25 = BM25Okapi(bm25_corpus)

# Runs bm25 for every query and stores output in file.

with open(run_file_path, 'w') as fo:
  for tid in tqdm(dev['Input.facet_id'].unique()):
    # print(tid)
    query = dev.loc[dev['Input.facet_id']==tid, 'Input.facet'].tolist()[0]
    topic = dev.loc[dev['Input.facet_id']==tid, 'Input.query'].tolist()[0]
    scores = bm25.get_scores(stem_tokenize(topic+' '+query))

    top_n = np.argsort(scores)[::-1][:100]
    scores = [scores[i] for i in top_n]
    # +' '+query
    bm25_ranked_list = bm25.get_top_n(stem_tokenize(topic, True),
                                    bm25_corpus,
                                    n=100)
    bm25_q_list = [' '.join(sent) for sent in bm25_ranked_list]
    # print(len(bm25_q_list))
    # print(len(doc_bank['tokenized_doc_str']))
    preds = [doc_bank['doc_id'][i] for i in top_n]
    # preds = doc_bank.set_index('tokenized_doc_str').loc[bm25_q_list, 'doc_id'].tolist()
    # print(len(preds))
    for i, qid in enumerate(preds):
      fo.write('{} 0 {} {} {} {} bm25\n'.format(tid, qid, i, len(preds)-i, scores[i]))

In [25]:
!pip install nltk


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [29]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [22]:
import pandas as pd
from rank_bm25 import BM25Okapi

from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
import numpy as np
import pickle
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')


# Define tokenization and stemming function
def stem_tokenize(text, remove_stopwords=True):
    stemmer = PorterStemmer()
    tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
    tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
    return [stemmer.stem(word) for word in tokens]

# File paths
request_file_path = './Final3_4o_mini.csv'  # Updated to the new file
doc_bank_path = './cwdocs12.tsv'
run_file_path = './dev_bm25_12_multiturn'

# Reads files and builds bm25 corpus (index)
dev = pd.read_csv(request_file_path, sep=',', engine='python', on_bad_lines='skip')
doc_bank = pd.read_csv(doc_bank_path, sep='\t').fillna('')

# Load the pre-tokenized documents
with open('./tokenized_12.pkl','rb') as f:
    tokenized_doc_str_list = pickle.load(f)

print(f"Length of tokenized_doc_str_list: {len(tokenized_doc_str_list)}")
print(f"Length of doc_bank: {len(doc_bank)}")

doc_bank['tokenized_doc_list']  = tokenized_doc_str_list
doc_bank['tokenized_doc_str'] = doc_bank['tokenized_doc_list'].map(lambda x: ' '.join(x))



bm25_corpus = doc_bank['tokenized_doc_list'].tolist()
bm25 = BM25Okapi(bm25_corpus)

# Function to combine topic and conversation history
def create_conversational_input(row):
    conversation = row['topic']
    # Concatenate all question-answer pairs, if they exist
    if not pd.isna(row['question1']) and not pd.isna(row['answer1']):
        conversation += ' ' + row['question1'] + ' ' + row['answer1']
    if not pd.isna(row['question2']) and not pd.isna(row['answer2']):
        conversation += ' ' + row['question2'] + ' ' + row['answer2']
    if not pd.isna(row['question3']) and not pd.isna(row['answer3']):
        conversation += ' ' + row['question3'] + ' ' + row['answer3']
    return conversation

# Add a new column for the conversational input
dev['conversational_input'] = dev.apply(create_conversational_input, axis=1)

# Runs bm25 for every query and stores output in a file.
with open(run_file_path, 'w') as fo:
    for tid in tqdm(dev['facet_id'].unique()):
        # Get the corresponding conversational input for the current topic
        conv_input = dev.loc[dev['facet_id'] == tid, 'conversational_input'].tolist()[0]
        # Tokenize the conversational input
        tokenized_query = stem_tokenize(conv_input)
        # Get BM25 scores for the tokenized query
        scores = bm25.get_scores(tokenized_query)

        # Retrieve the top 100 documents based on the BM25 scores
        top_n = np.argsort(scores)[::-1][:100]
        scores = [scores[i] for i in top_n]
        bm25_ranked_list = bm25.get_top_n(tokenized_query, bm25_corpus, n=100)

        # Get document ids for the top results
        preds = [doc_bank['doc_id'][i] for i in top_n]
        
        # Write the output in TREC format
        for i, qid in enumerate(preds):
            fo.write('{} 0 {} {} {} {} bm25\n'.format(tid, qid, i, len(preds)-i, scores[i]))


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


Length of tokenized_doc_str_list: 9918
Length of doc_bank: 9918


  0%|          | 0/282 [00:00<?, ?it/s]


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/kimia/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [23]:
import pandas as pd
import csv

In [44]:
request_file_path = './Final2_4o_mini.csv'

df = pd.read_csv(request_file_path)

df.head()


,facet,facet_id,topic_id,topic,question1,answer1,question_id1,image1_1,image1_2,image1_3,...,question_id3,image3_1,image3_2,image3_3,question4,answer4,question_id4,image4_1,image4_2,image4_3
0,What are some of the causes for a lump in the ...,F0548,237,lump in throat,would you like to know how to fix a lump in th...,I'm more interested in understanding the reaso...,Q03480,237_Q03480_3495_1.jpg,237_Q03480_3495_2.jpg,237_Q03480_3495_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Find information about the Panthers sports tea...,F0236,159,porterville,are you looking for a hotel near porterville,"I'm interested in sports teams, but that's not...",Q00630,159_Q00630_1853_1.jpg,159_Q00630_1853_2.jpg,159_Q00630_1853_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Take me to the homepage of the College Board, ...",F1064,98,sat,would you like to know about the most recent s...,I'm thinking about something related to educat...,Q03426,98_Q03426_2747_1.jpg,98_Q03426_2747_2.jpg,98_Q03426_2747_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,information on ground elevation changes and mo...,F0190,149,uplift at yellowstone national park,may i tell you the numnber of earthquakes reco...,"Well, I can't pinpoint the exact details regar...",Q02652,149_Q02652_1148_1.jpg,149_Q02652_1148_2.jpg,149_Q02652_1148_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What causes the heart to beat faster or slower?,F0624,26,lower heart rate,are you planning to run or exercise,I'm thinking about how different activities mi...,Q01193,26_Q01193_697_1.jpg,26_Q01193_697_2.jpg,26_Q01193_697_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
sorted_df = df.sort_values(by='facet_id')
df_reset = sorted_df.reset_index(drop=True)
df_reset.head()

,facet,facet_id,topic_id,topic,question1,answer1,question_id1,image1_1,image1_2,image1_3,...,question_id3,image3_1,image3_2,image3_3,question4,answer4,question_id4,image4_1,image4_2,image4_3
0,"""Find the TIME magazine photo essay """"Barack O...",F0001,1,obama family tree,Would you like to see the family tree with pic...,I'm really interested in visuals for this.,Q05588,1_Q05588_1646_1.jpg,1_Q05588_1646_2.jpg,1_Q05588_1646_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Find the TIME magazine photo essay """"Barack O...",F0001,1,obama family tree,Would you like to see the family tree with pic...,I have a preference for visuals over text when...,Q05588,1_Q05588_1646_1.jpg,1_Q05588_1646_2.jpg,1_Q05588_1646_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Find the TIME magazine photo essay """"Barack O...",F0001,1,obama family tree,Are you looking for information about the Obam...,I'm interested in exploring some specific cont...,Q05592,1_Q05592_1650_1.jpg,1_Q05592_1650_2.jpg,1_Q05592_1650_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Find the TIME magazine photo essay """"Barack O...",F0001,1,obama family tree,Are you looking for information about the Obam...,"I can't confirm that, but I am definitely inte...",Q05592,1_Q05592_1650_1.jpg,1_Q05592_1650_2.jpg,1_Q05592_1650_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Find the TIME magazine photo essay """"Barack O...",F0001,1,obama family tree,Would you like to see the family tree with pic...,I'm more interested in a visual representation...,Q05588,1_Q05588_1646_1.jpg,1_Q05588_1646_2.jpg,1_Q05588_1646_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_reset.to_csv('./sorted_final2_4o_mini.csv', index=True)

In [49]:
df= pd.read_csv("./sorted_final4_4o_mini.csv")
df.head()

,id,facet,facet_id,topic_id,topic,question1,answer1,id_1,image1_1,image1_2,...,id_3,image3_1,image3_2,image3_3,question4,answer4,id_4,image4_1,image4_2,image4_3
0,0,Do any internet providers still sell dial-up?,F0005,10,cheap internet,are you wondering where cheap internet is,"No, that’s not what I’m looking for.",Q01453,10_Q01453_944_1.jpg,10_Q01453_944_2.jpg,...,Q05638,10_Q05638_1696_1.jpg,10_Q05638_1696_2.jpg,10_Q05638_1696_3.jpg,Would you like news on cheap internet?,yes i need to know if any of them still have d...,Q05642,10_Q05642_1700_1.jpg,10_Q05642_1700_2.jpg,10_Q05642_1700_3.jpg
1,1,Find a site where I can determine room price a...,F0011,101,ritz carlton lake las vegas,Are you interested in the price of the Ritz Ca...,"No, I am not interested in that.",Q04390,101_Q04390_448_1.jpg,101_Q04390_448_2.jpg,...,Q03582,101_Q03582_2711_1.jpg,101_Q03582_2711_2.jpg,101_Q03582_2711_3.jpg,Are you interested in seeing the hotel room th...,"I appreciate the suggestion, but I'm really lo...",Q04388,101_Q04388_446_1.jpg,101_Q04388_446_2.jpg,101_Q04388_446_3.jpg
2,2,Find information about C. J. Walker's involvem...,F0021,103,madam cj walker,do you want to know what madam cj walker invented,"No, I’m not interested in that topic.",Q02290,103_Q02290_1055_1.jpg,103_Q02290_1055_2.jpg,...,Q03397,103_Q03397_3092_1.jpg,103_Q03397_3092_2.jpg,103_Q03397_3092_3.jpg,do you want to know what madam cj walker is fa...,No. I want to find information about C.J. Walk...,Q02291,103_Q02291_2998_1.jpg,103_Q02291_2998_2.jpg,103_Q02291_2998_3.jpg
3,3,"What was the 2008 budget for Cass County, MO?",F0032,107,cass county missouri,would you like to know more about the city,"No, I am not particularly interested in that t...",Q03544,107_Q03544_37_1.jpg,107_Q03544_37_2.jpg,...,Q03051,107_Q03051_2988_1.jpg,107_Q03051_2988_2.jpg,107_Q03051_2988_3.jpg,Are you interested in government offcials from...,No. I would like to know what the 2008 budget ...,Q04125,107_Q04125_183_1.jpg,107_Q04125_183_2.jpg,107_Q04125_183_3.jpg
4,4,I'm looking for some free sample GMAT exams to...,F0045,11,gmat prep classes,do you want to know a list of top gmat prep cl...,"No, that's not what I'm looking for.",Q01962,11_Q01962_640_1.jpg,11_Q01962_640_2.jpg,...,Q00006,11_Q00006_1983_1.jpg,11_Q00006_1983_2.jpg,11_Q00006_1983_3.jpg,Do you also want information on GRE preparation?,Yes but I only need free sample GMAT exams to ...,Q04748,11_Q04748_806_1.jpg,11_Q04748_806_2.jpg,11_Q04748_806_3.jpg


In [1]:
import pandas as pd

In [ ]:
# retrieve top10 documents given the topic
# import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
import numpy as np
# nltk.download('punkt')
# nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
    stemmer = PorterStemmer()
    tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
    tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
    return [stemmer.stem(word) for word in tokens]


# Files paths
request_file_path = './newfiles/sorted_final2_4o_mini.csv'
doc_bank_path_09 = '../output.tsv'
doc_bank_path_12 = './cwdocs12.tsv'
run_file_path = './first_turn/bm25_first_turn_2'

# Reads files and build bm25 corpus (index)

dev = pd.read_csv(request_file_path, sep=',', engine='python', on_bad_lines='skip')
doc_bank_12 = pd.read_csv(doc_bank_path_12, sep='\t').fillna('')
doc_bank_09 = pd.read_csv(doc_bank_path_09, sep='\t').fillna('')

# doc_bank = {}
# doc_bank['tokenized_doc_list'] = doc_bank['doc'].map(stem_tokenize)
# tokenized_doc_str_list = []
# for d in tqdm(doc_bank['doc']):
#     tokenized_doc_str = stem_tokenize(d)
#     tokenized_doc_str_list.append(tokenized_doc_str)

def create_conversational_input(row):
    conversation = row['topic']  # Start with the main topic
    # Concatenate all question-answer pairs, if they exist
    if not pd.isna(row['question1']) and not pd.isna(row['answer1']):
        conversation += ' ' + row['question1'] + ' ' + row['answer1']
    # if not pd.isna(row['question2']) and not pd.isna(row['answer2']):
    #     conversation += ' ' + row['question2'] + ' ' + row['answer2']
    # if not pd.isna(row['question3']) and not pd.isna(row['answer3']):
    #     conversation += ' ' + row['question3'] + ' ' + row['answer3']
    # if not pd.isna(row['question4']) and not pd.isna(row['answer4']):
    #     conversation += ' ' + row['question4'] + ' ' + row['answer4']
    return conversation

# Add a new column for the conversational input
dev['conversational_input'] = dev.apply(create_conversational_input, axis=1)

import pickle
with open('../corpus_tokenized.pkl','rb') as f:
    tokenized_doc_str_list_09 = pickle.load(f)
with open('./tokenized_12.pkl','rb') as f:
    tokenized_doc_str_list_12 = pickle.load(f)
# with open('tokenized_12.pkl','wb') as f:
#     pickle.dump(tokenized_doc_str_list,f,protocol=1)
print(len(tokenized_doc_str_list_09))
print(len(doc_bank_09))
doc_bank_09['tokenized_doc_list_09']  = tokenized_doc_str_list_09
doc_bank_09['tokenized_doc_str_09'] = [' '.join(x) for x in doc_bank_09['tokenized_doc_list_09']]
doc_bank_12['tokenized_doc_list_12']  = tokenized_doc_str_list_12
doc_bank_12['tokenized_doc_str_12'] = [' '.join(x) for x in doc_bank_12['tokenized_doc_list_12']]
bm25_corpus_09 = doc_bank_09['tokenized_doc_list_09'].tolist()
bm25_09 = BM25Okapi(bm25_corpus_09)
bm25_corpus_12 = doc_bank_12['tokenized_doc_list_12'].tolist()
bm25_12 = BM25Okapi(bm25_corpus_12)

# Runs bm25 for every query and stores output in file.
with open(run_file_path, 'w') as fo:
    rowid=-1
    for row in tqdm(dev.to_dict(orient='records')):
        tid = row['facet_id']
        id = row['topic_id']
        rowid+=1

        if int(id) <=200:
            bm25 = bm25_09
            bm25_corpus = bm25_corpus_09
            doc_bank = doc_bank_09
        else:
            bm25 = bm25_12
            bm25_corpus = bm25_corpus_12
            doc_bank = doc_bank_12

        # print(query)
        # print(question)
        # print(answer)
        conv_input = row['conversational_input']
        # print(conv_input)
        # raise RuntimeError
    
        
        tokenized_query = stem_tokenize(conv_input)
        
        scores = bm25.get_scores(tokenized_query)
        top_n = np.argsort(scores)[::-1][:100]
        scores = [scores[i] for i in top_n]
        bm25_ranked_list = bm25.get_top_n(stem_tokenize(conv_input , True),
                                          bm25_corpus,
                                          n=100)
        # print(scores)
        preds = [doc_bank['doc_id'][i] for i in top_n]
        for i, qid in enumerate(preds):
            fo.write('{} {} 0 {} {} {} {}bm25\n'.format(rowid,tid, qid, i, len(preds)-i, scores[i]))

In [2]:
!pip install rank_bm25

In [3]:
file1 = './full_turn/bm25_full_turn_2'
file2 = './full_turn/bm25_full_turn_3'
file3 = './full_turn/bm25_full_turn_4'

# Filenames of the text files
files = [file1, file2, file3]

# Open a new file to write the concatenated content
with open('./full_turn/full_bm25_full_turn.txt', 'w', encoding='utf-8') as outfile:
    for filename in files:
        with open(filename, 'r', encoding='utf-8') as infile:
            outfile.write(infile.read())
            outfile.write('\n')  # Optional: Add a newline between files

In [ ]:
# Read from the input text file
with open('./full_turn/full_bm25_full_turn.txt', 'r', encoding='utf-8') as infile:
    lines = infile.readlines()

# Processed data storage
processed_data = []

# Process each line
for line in lines:
    parts = line.split()
    if len(parts) < 7:  # Ensure there are enough columns
        continue
    new_second_column = parts[1][1:]  # Remove the 'F'
    third_column = "Q" + parts[2]  # Add 'Q' to the third column
    processed_line = f"{new_second_column} {third_column} {' '.join(parts[3:])}"
    processed_data.append(processed_line)


# Write the sorted data to the output text file
with open('./full_turn/bm25_full_full_turn.txt', 'w', encoding='utf-8') as outfile:
    for line in processed_data:
        outfile.write(line + '\n')

In [4]:
res.head()

,facet,facet_id,topic_id,topic,question1,answer1,question_id1,image1_1,image1_2,image1_3,...,question4,answer4,question_id4,image4_1,image4_2,image4_3,id_1,id_2,id_3,id_4
0,What are some of the causes for a lump in the ...,F0548,237,lump in throat,would you like to know how to fix a lump in th...,I'm more interested in understanding the reaso...,Q03480,237_Q03480_3495_1.jpg,237_Q03480_3495_2.jpg,237_Q03480_3495_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Find information about the Panthers sports tea...,F0236,159,porterville,are you looking for a hotel near porterville,"I'm interested in sports teams, but that's not...",Q00630,159_Q00630_1853_1.jpg,159_Q00630_1853_2.jpg,159_Q00630_1853_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Take me to the homepage of the College Board, ...",F1064,98,sat,would you like to know about the most recent s...,I'm thinking about something related to educat...,Q03426,98_Q03426_2747_1.jpg,98_Q03426_2747_2.jpg,98_Q03426_2747_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,information on ground elevation changes and mo...,F0190,149,uplift at yellowstone national park,may i tell you the numnber of earthquakes reco...,"Well, I can't pinpoint the exact details regar...",Q02652,149_Q02652_1148_1.jpg,149_Q02652_1148_2.jpg,149_Q02652_1148_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What causes the heart to beat faster or slower?,F0624,26,lower heart rate,are you planning to run or exercise,I'm thinking about how different activities mi...,Q01193,26_Q01193_697_1.jpg,26_Q01193_697_2.jpg,26_Q01193_697_3.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Open the input and output files
with open('all.qrel', 'r') as infile, open('qrelmodif.txt', 'w') as outfile:
    for line in infile:
        parts = line.split()
        # Remove the first '0'
        # Eliminate 'F' from the second part
        parts[0] = parts[0][1:]
        # Add 'Q' to the third part
        # Write the modified line to the output file
        outfile.write(' '.join(parts) + '\n')


ModuleNotFoundError: No module named 'datasets'

In [1]:
!pip install ir_datasets


In [3]:
import ir_datasets
dataset = ir_datasets.load("clueweb09")
for doc in dataset.docs_iter():
    print(doc)
    break

[INFO] ClueWeb09 is available by hard drives from CMU here: <https://lemurproject.org/clueweb09/>
More details about the procedure can be found here: <https://ir-datasets.com/ClueWeb09.html#DataAccess>.
Link the ClueWeb09 source files here: /Users/alirezaamiri/.ir_datasets/clueweb09/corpus
Should contain directories like ClueWeb09_English_1


FileNotFoundError: /Users/alirezaamiri/.ir_datasets/clueweb09/corpus

In [8]:
# Read the facet IDs from test.qrel
with open("test.qrel", "r") as qrel_file:
    qrel_lines = qrel_file.readlines()

# Extract unique facet IDs
facet_ids = set(line.split()[0][1:] for line in qrel_lines)
print(sorted(facet_ids))

# Read and filter the modified_dataset2.txt
with open("modified_dataset2.txt", "r") as dataset_file:
    dataset_lines = dataset_file.readlines()

# Filter the dataset lines
filtered_lines = [
    line for line in dataset_lines if line.split()[0] in facet_ids
]

for line in dataset_lines:
    if line.split()[0] in facet_ids:
        print(line.split()[0])
        break
    print(line.split()[0])
print("gooz")

# Write the filtered lines to a new file or print them
with open("filtered_dataset.txt", "w") as output_file:
    output_file.writelines(filtered_lines)

# Alternatively, print the filtered lines
# for line in filtered_lines:
#     print(line)

['0017', '0018', '0019', '0050', '0056', '0061', '0066', '0068', '0075', '0076', '0096', '0100', '0111', '0114', '0116', '0120', '0121', '0126', '0135', '0153', '0173', '0176', '0183', '0184', '0198', '0207', '0208', '0213', '0225', '0232', '0249', '0250', '0254', '0255', '0259', '0260', '0261', '0264', '0280', '0290', '0302', '0314', '0351', '0381', '0407', '0420', '0425', '0449', '0454', '0470', '0477', '0482', '0497', '0523', '0534', '0553', '0562', '0578', '0585', '0606', '0629', '0634', '0637', '0644', '0653', '0657', '0663', '0666', '0683', '0698', '0701', '0740', '0741', '0755', '0757', '0758', '0761', '0766', '0768', '0774', '0776', '0778', '0794', '0803', '0806', '0818', '0826', '0830', '0842', '0848', '0866', '0868', '0899', '0907', '0908', '0921', '0935', '0944', '0953', '0959', '0973', '0980', '0985', '0991', '1026', '1057', '1066']
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
0001
000

In [13]:
def convert_format(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    with open(output_file, 'w') as f:
        for line in lines:
            parts = line.split()
            # Modify the format and remove the first F
            new_line = f"{parts[1][1:]} Q{parts[2]} {' '.join(parts[3:])}\n"
            f.write(new_line)

# Use the function
convert_format('./200bm/human-dev-llmanswers-2turn_200', './200bm/modified-human-dev-llmanswers-2turn_200.txt')